In [1]:
""" Create dialogue corpus for movies based on :"""

' Create dialogue corpus for movies based on :'

In [1]:
from convokit import Corpus, download
import pandas as pd
import os
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np

from importlib import reload
import  helper_functions as hf
reload(hf)

<module 'helper_functions' from 'c:\\Users\\Me\\Courses\\Spiced_DS\\git_repos\\finalproject\\helper_functions.py'>

In [2]:
# download data
data = Corpus(filename=download("movie-corpus"))

In [3]:
# create dataframes 
conversations = data.get_conversations_dataframe()
utterances = data.get_utterances_dataframe()
utterances = utterances.rename(columns={'meta.movie_id':'meta.movie_idx'})

In [4]:
print(conversations.head(5))
print(utterances.head(5))

      vectors meta.movie_idx             meta.movie_name meta.release_year  \
id                                                                           
L1044      []             m0  10 things i hate about you              1999   
L984       []             m0  10 things i hate about you              1999   
L924       []             m0  10 things i hate about you              1999   
L870       []             m0  10 things i hate about you              1999   
L866       []             m0  10 things i hate about you              1999   

      meta.rating meta.votes             meta.genre  
id                                                   
L1044        6.90      62847  ['comedy', 'romance']  
L984         6.90      62847  ['comedy', 'romance']  
L924         6.90      62847  ['comedy', 'romance']  
L870         6.90      62847  ['comedy', 'romance']  
L866         6.90      62847  ['comedy', 'romance']  
      timestamp          text speaker reply_to conversation_id meta.movie_i

In [ ]:
print("Movies contained in this dataset:")
print(conversations['meta.movie_name'].unique())

In [ ]:
# combine all texts/quotes of a movie together
scripts = utterances.groupby('meta.movie_idx')['text'].unique().str.join(' ')
scripts = pd.DataFrame(scripts)
# select relevant columns
conversations = conversations[['meta.movie_idx','meta.movie_name']].set_index('meta.movie_idx')
# combine texts and movies
corpus = scripts.join(conversations, on='meta.movie_idx').drop_duplicates()
corpus = corpus.rename(columns={'meta.movie_name':'movie'})
corpus.index.name='idx'

In [ ]:
print(corpus)

In [ ]:
corpus_cleaned = hf.clean_dialogue_corpus(corpus)
corpus_cleaned = hf.clean_movie_titles(corpus_cleaned)

In [ ]:
# save cleaned corpus to csv file
os.makedirs('./output/', exist_ok=True)  
#corpus_cleaned.to_csv('./output/dialogue_corpus.csv', index=True)  

In [12]:
# sentiment analysis
utterances_sentiment = utterances.copy()
utterances_sentiment['sentiment'] = np.nan
utterances_sentiment

,timestamp,text,speaker,reply_to,conversation_id,meta.movie_idx,meta.parsed,vectors,sentiment
id,,,,,,,,,
L1045,None,They do not!,u0,L1044,L1044,m0,"[{'rt': 1, 'toks': [{'tok': 'They', 'tag': 'PR...",[],NaN
L1044,None,They do to!,u2,None,L1044,m0,"[{'rt': 1, 'toks': [{'tok': 'They', 'tag': 'PR...",[],NaN
L985,None,I hope so.,u0,L984,L984,m0,"[{'rt': 1, 'toks': [{'tok': 'I', 'tag': 'PRP',...",[],NaN
L984,None,She okay?,u2,None,L984,m0,"[{'rt': 1, 'toks': [{'tok': 'She', 'tag': 'PRP...",[],NaN
L925,None,Let's go.,u0,L924,L924,m0,"[{'rt': 0, 'toks': [{'tok': 'Let', 'tag': 'VB'...",[],NaN
...,...,...,...,...,...,...,...,...,...
L666371,None,Lord Chelmsford seems to want me to stay back ...,u9030,L666370,L666369,m616,"[{'rt': 2, 'toks': [{'tok': 'Lord', 'tag': 'NN...",[],NaN
L666370,None,I'm to take the Sikali with the main column to...,u9034,L666369,L666369,m616,"[{'rt': 1, 'toks': [{'tok': 'I', 'tag': 'PRP',...",[],NaN
L666369,None,"Your orders, Mr Vereker?",u9030,None,L666369,m616,"[{'rt': 1, 'toks': [{'tok': 'Your', 'tag': 'PR...",[],NaN


In [13]:
# run sentiment analysis-vader
s  = SentimentIntensityAnalyzer()
for idx, line in enumerate(utterances_sentiment['text']):
    sentiment = s.polarity_scores(line)
    score = sentiment['compound']
    utterances_sentiment.loc[idx,'sentiment'] = score
    #print(news)
    #print(sentiment)
